查询索引


安装llama-index和databricks-vectorsearch。您必须在Databricks运行时中才能使用Vector Search python客户端。


In [ ]:
# Databricks向量搜索

Databricks向量搜索是一个内置于Databricks Intelligence Platform中的向量数据库，与其治理和生产力工具集成在一起。完整文档请参见：https://docs.databricks.com/en/generative-ai/vector-search.html


导入databricks依赖项


In [ ]:
from databricks.vector_search.client import (
    VectorSearchIndex,
    VectorSearchClient,
)

导入LlamaIndex依赖项


In [ ]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    ServiceContext,
    StorageContext,
)
from llama_index.vector_stores.databricks import DatabricksVectorSearch

加载示例数据


In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

读取数据


In [ ]:
# 加载文档documents = SimpleDirectoryReader("./data/paul_graham/").load_data()print(f"总文档数：{len(documents)}")print(f"第一个文档，id：{documents[0].doc_id}")print(f"第一个文档，哈希值：{documents[0].hash}")print(    "第一个文档，文本"    f"（{len(documents[0].text)} 个字符）：\n{'='*20}\n{documents[0].text[:360]} ...")

创建一个Databricks Vector Search端点，用于提供索引服务。


In [ ]:
# 创建一个向量搜索端点client = VectorSearchClient()client.create_endpoint(    name="llamaindex_dbx_vector_store_test_endpoint", endpoint_type="STANDARD")

创建Databricks Vector Search索引，并从文档中构建索引。


In [ ]:
# 创建一个向量搜索索引# 必须放置在启用了Unity Catalog的模式内# 我们将使用自管理的嵌入（即由LlamaIndex管理）而不是由Databricks管理的索引databricks_index = client.create_direct_access_index(    endpoint_name="llamaindex_dbx_vector_store_test_endpoint",    index_name="my_catalog.my_schema.my_test_table",    primary_key="my_primary_key_name",    embedding_dimension=1536,  # 与您将要使用的嵌入模型维度相匹配    embedding_vector_column="my_embedding_vector_column_name",  # 您可以随意命名此列 - 它将被LlamaIndex类捕获    schema={        "my_primary_key_name": "string",        "my_embedding_vector_column_name": "array<double>",        "text": "string",  # 一个列必须与下面创建的DatabricksVectorSearch实例中的text_column匹配；这将保存原始节点文本，        "doc_id": "string",  # 一个列必须包含参考文档ID（这将由LlamaIndex自动填充）        # 添加您可能在节点中拥有的任何其他元数据（Databricks Vector Search支持元数据过滤）        # 请注意，这些字段必须显式添加才能用于元数据过滤    },)databricks_vector_store = DatabricksVectorSearch(    index=databricks_index,    text_column="text",    columns=None,  # 您还必须在此记录您的元数据字段名称)  # text_column对于自管理的嵌入是必需的storage_context = StorageContext.from_defaults(    vector_store=databricks_vector_store)index = VectorStoreIndex.from_documents(    documents, storage_context=storage_context)

查询索引


In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("Why did the author choose to work on AI?")

print(response.response)